In [0]:
from google.colab import drive
from google.colab import files

In [0]:
'''
To use the functionality in this cell:
  Select 'Runtime'
  Select 'Change Runtime Type'
  Select 'Python 3', 'GPU', and check 'Omit cell output when saving this notebook'
'''

#Install and import required packages
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

#Get memory footprint of hosted GPU
GPUs = GPU.getGPUs()

#Only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]

#Print available RAM
def printm():
 process = psutil.Process(os.getpid())
 print()
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

printm()

In [0]:
#Read and write files directly from google drive
drive.mount('/content/drive', force_remount=True)

In [0]:
!pip install rasterio
!pip install geopandas
!apt-get install libgeos-3.5.0
!apt-get install libgeos-dev
!pip install https://github.com/matplotlib/basemap/archive/master.zip
!pip install pyproj==1.9.6

In [0]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from mpl_toolkits.basemap import Basemap
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
from matplotlib.colors import LightSource, LinearSegmentedColormap
import rasterio
import rasterio.plot as rplot
from rasterio.warp import calculate_default_transform, reproject, Resampling
import geopandas as gpd
import numpy as np
from descartes import PolygonPatch
import fiona

In [0]:
a_vect = gpd.read_file('drive/My Drive/Hamish mapping/Hamish mapping/useful general mapping data - elevation and perimeter/Linea_Costa/PERFIL_Project.shp', driver='ESRI Shapefile')
a_dem = rasterio.open('drive/My Drive/Hamish mapping/Hamish mapping/Environmental layers/DEM/Elevation.tif')
#a_vect = gpd.read_file('drive/My Drive/Hamish mapping/Hamish mapping/Environmental layers/Shp/Island_volcanos_wgs.shp', driver='ESRI Shapefile')

In [0]:
#Check crs of each dataset
print(a_vect.crs)
print(a_dem.crs)

In [0]:
a_dem_data = a_dem.read()
type(a_dem_data)

In [0]:
#losing spatial reference here
dem_meta = a_dem.profile

#Read data from rasterio reader, read the first (and only) band
a_dem_data = a_dem.read()[0]

#Replace file NA value with np.nan
a_dem_data[a_dem_data==a_dem.nodatavals[0]] = np.nan

#Replace 0 elevation values with np.nan
a_dem_data[a_dem_data==0] = np.nan

with rasterio.open('drive/My Drive/Hamish mapping/Hamish mapping/tmp/DEM_nan.tif', 'w', **dem_meta) as dst:
    dst.write(a_dem_data, 1)
    
a_dem_data = rasterio.open('drive/My Drive/Hamish mapping/Hamish mapping/tmp/DEM_nan.tif')

In [0]:
#create a hillshade layer fromt he elevation dem
ls = LightSource(azdeg=315, altdeg=45)
cmap = plt.cm.gist_earth
hillshade = ls.hillshade(a_dem_data.read(1), vert_exag=1, dx=a_dem_data.width, dy=a_dem_data.height, fraction=1.0)

with rasterio.open('drive/My Drive/Hamish mapping/Hamish mapping/tmp/hillshade.tif', 'w', **dem_meta) as dst:
    dst.write(hillshade, 1)
    
hillshade = rasterio.open('drive/My Drive/Hamish mapping/Hamish mapping/tmp/hillshade.tif')

In [0]:
#truncate the gist_earth colormap to take out blue values
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

#color references:
#https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html
#https://matplotlib.org/3.1.0/gallery/color/named_colors.html

In [0]:
#New cmap
cmap = plt.get_cmap('gist_earth')
new_cmap = truncate_colormap(cmap, 0.4, 1.0)

In [0]:
#make linwidth thicker, put under the raster
plt.rcParams["figure.figsize"] = [20,10]
ax = a_vect.geometry.plot(color='k', linewidth=0.75)

ax.grid(True, color='k', alpha=0.3)
ax.set_facecolor('lightblue')

#plot hillshade layer
ax.imshow(hillshade.read(1), 
          extent=(a_dem_data.bounds[0], a_dem_data.bounds[2], a_dem_data.bounds[1], a_dem_data.bounds[3]),
          cmap='gray')

#plot terrain color layer
ax.imshow(a_dem_data.read(1), 
          extent=(a_dem_data.bounds[0], a_dem_data.bounds[2], a_dem_data.bounds[1], a_dem_data.bounds[3]),
          cmap=new_cmap,
          alpha=0.6)

In [0]:
#Simple layout here

In [0]:
#Bounding box analysis in new file

In [0]:
#Check with J about colors, hillshading, and terrain colors
#add additional information, scale bar, north arrow, credits

#create map with 'subtle' theme for plotting other raster data

#Wrap everything into a class - i.e. colormap(island, etc.)
#create class method, colormap.add_data(data, etc.)

#island bbox - by volcano dataset